# Сравнительный анализ линтеров Python  
**Ruff vs Flake8 vs Pylint**

---

## Важность линтинга
Линтинг — важный этап в современной разработке на Python, который помогает:  

-  Обнаруживать ошибки на ранних стадиях  
-  Поддерживать единый стиль кода  
-  Улучшать читаемость и поддерживаемость  
-  Сокращать время на ревью кода  

---

## Инструменты для сравнения

### Flake8
- Классический линтер с **модульной архитектурой**.  
- Расширяется **плагинами** для дополнительной функциональности.
  
### Ruff
- Современный линтер, написанный на **Rust**.  
- Объединяет функциональность множества инструментов в одном **быстром пакете**.

### Pylint
- Самый строгий инструмент **статического анализа**.  
- Глубокий анализ кода с акцентом на **поиск багов и проблем архитектуры**.

---

## Цели сравнения
-  Проанализировать **производительность** инструментов  
-  Сравнить **качество обнаружения ошибок**  
-  Оценить **гибкость настройки**  

---

## Замечание
Мы будем использовать NBQA .NBQA — инструмент для запуска Python-линтеров и форматтеров в Jupyter Notebook (.ipynb). Jupyter Notebook хранит код в JSON-формате, поэтому линтеры не могут работать с .ipynb файлами напрямую.
### **Принцип работы NBQA**
- Извлекает код из ячеек notebook'а
- Создает временные .py файлы
- Запускает выбранный инструмент на этих файла
- Применяет изменения обратно в исходный .ipynb файл

---

## Плохой код:

In [4]:
import os, sys
import sympy as sp

def  bad_function ( ) :
    a=1
    b=2
    print(a+b)
    return

unused_var = 10

def long_line():
    print("This is a very long line that definitely exceeds the typical character limit for most linters and should trigger warnings")

# Подробный разбор Flake8 + плагины

Flake8 — классический линтер для Python с модульной архитектурой, который расширяется плагинами для дополнительной функциональности.Flake8 — давно устоявшийся инструмент, объединяющий **PyFlakes** (анализ кода на ошибки) и **pycodestyle** (проверка PEP8-стиля), а также скрипт **McCabe** (анализ сложности). Flake8 запускается одной командой и выводит список предупреждений по файлам.


## Преимущества
- Зрелость и стабильность
- Модульный подход
- Огромная экосистема плагинов
- Активное сообщество
- Гибкость настройки
- Поддержка пользовательских правил

## Недостатки
- Сложность конфигурации при множестве плагинов
- Нет автоматического исправления
- Flake8 в базовой конфигурации использует один поток, хотя есть флаг -j для мультипроцессности. При большом количестве файлов и плагинов скорости может не хватать (особенно на медленных CI-агентах)

## Установка и настройка
Сначала мы устанавливаем flake8 и плагины
```bash
pip install flake8 flake8-bugbear flake8-docstrings flake8-comprehensions
```
В корне проекта создаём **setup.cfg**, если его ещё нет, и добавляем секцию для Ruff:
```bash
[tool.flake8]
max-line-length = 88
select = ["E", "F", "W", "B", "D"]  # E,F,W = стандартные, B = bugbear, D = docstrings
ignore = ["E203", "W503"] # Допустим игнорируем данные ошибки
exclude = [".git", "__pycache__"]
max-complexity = 10
docstring-convention = "google"  # flake8-docstrings
```

- **pyflakes** – анализирует код на логические ошибки (неиспользуемые импорты и переменные, неопределённые имена).
- **pycodestyle** – проверяет соблюдение стиля кода по PEP8 (отступы, пробелы, длина строк, пустые строки).
- **mccabe** – вычисляет цикломатическую сложность функций и предупреждает, если она слишком высока.
- **flake8-bugbear** – находит потенциально опасные или подозрительные конструкции в коде (например, изменяемые значения по умолчанию).
- **flake8-comprehensions** – оптимизирует использование генераторов и comprehensions (подсказывает, когда запись можно упростить).
- **flake8-docstrings** – проверяет наличие и оформление docstring в модулях, классах и функциях.

## Описание ошибок
- E (Error) — ошибки стиля PEP8 (pycodestyle). Например: лишние пробелы, неверные отступы, слишком длинные строки.
- W (Warning) — предупреждения стиля PEP8. Например: пустые строки с пробелами, нежелательные пробелы перед скобками.
- F (Flake8 / Pyflakes) — логические ошибки или потенциальные баги. Например: неиспользуемые импорты, неопределённые переменные.
- C (Cyclomatic / Complexity) — цикломатическая сложность (от mccabe). Если функция слишком сложная, выдаёт предупреждение.
- COM (Comprehensions) — рекомендации по оптимизации генераторов и comprehensions.
- B	(Bugbear) — ошибки и подозрительные конструкции, найденные плагином flake8-bugbear.
- D (Docstrings) — ошибки стиля и отсутствия docstring

## Использование
```bash 
flake8 path/to/file #для запуска из консоли
!nbqa flake8 path/to/file #если хотите использовать внутри jupyter-notebook
```

## Пример

In [1]:
!nbqa flake8 Linters.ipynb

Linters.ipynb:cell_1:0:1: D100 Missing docstring in public module
Linters.ipynb:cell_1:1:1: F401 'os' imported but unused
Linters.ipynb:cell_1:1:1: F401 'sys' imported but unused
Linters.ipynb:cell_1:1:10: E401 multiple imports on one line
Linters.ipynb:cell_1:2:1: F401 'sympy as sp' imported but unused
Linters.ipynb:cell_1:5:1: D103 Missing docstring in public function
Linters.ipynb:cell_1:5:4: E271 multiple spaces after keyword
Linters.ipynb:cell_1:5:18: E211 whitespace before '('
Linters.ipynb:cell_1:5:20: E201 whitespace after '('
Linters.ipynb:cell_1:5:22: E203 whitespace before ':'
Linters.ipynb:cell_1:6:6: E225 missing whitespace around operator
Linters.ipynb:cell_1:7:6: E225 missing whitespace around operator
Linters.ipynb:cell_1:15:1: D103 Missing docstring in public function
Linters.ipynb:cell_1:16:80: E501 line too long (134 > 79 characters)


## Рассмотрим вывод
- D100 – отсутствует docstring в модуле
- D103 – отсутствует docstring в функции
- F401 – импортированная переменная не используется (os, sys, sympy)
- E401 – несколько импортов на одной строке
- E271 – лишние пробелы после ключевого слова
- E211 – лишний пробел перед скобкой
- E201 – пробел после открывающей скобки
- E203 – пробел перед двоеточием
- E225 – отсутствует пробел вокруг оператор
- E501 – строка слишком длинная (>79 символов)


# Подробный разбор Ruff
Ruff — современный линтер для Python, написанный на Rust, который объединяет функциональность множества инструментов в одном быстром пакете.
- Написан на **Rust** для максимальной производительности ([120 тысяч строк кода за 0.2 секунды](https://pythonspeed.com/articles/pylint-flake8-ruff/#:~:text=Benchmarking%20three%20alternatives,one%20single%20lint%3A))
- Подход **"всё-в-одном"** — замена Flake8, isort, pyupgrade и других

## Преимущества Ruff:
- Производительность: Ruff настолько быстр, что может выполнять автопочинку (--fix) и форматирование кода почти без замедления. Кроме того, инструмент поддерживает кэширование, чтобы не анализировать неизменённые файлы повторно
- Ruff совмещает функции линтера и форматтера (он может работать как замена и Flake8, и Black/Isort одновременно)

## Недостатки Ruff:
- Относительная новизна: могут быть баги, которые еще не обнаружили
- Не поддерживает пользовательские правила
- Меньше глубины анализа, чем у Pylint


## Установка и настройка
```bash
pip install ruff
```

В корне проекта создаём **pyproject.toml**, если его ещё нет, и добавляем секцию для Ruff:
```bash
[tool.ruff]
select = ["E", "F", "W"]
ignore = ["E501"]
exclude = [".git", "tests"] 
```
Ruff поддерживает **те же коды, что Flake8**, потому что он эмулирует плагины.
Но дополнительно есть:
- UP* — подсказки от pyupgrade (например, использование нового синтаксиса Python).
- ISC* — ошибки форматирования строк (из flake8-implicit-str-concat).
- TID* — проблемы с импортами (из flake8-tidy-imports).
- ANN* — аннотации типов (flake8-annotations).
## Использование 
```bash
ruff check /path/to/file #для проверки
ruff check /path/to/file --fix #для проверки и исправления ошибок
!nbqa ruff check /path/to/file #если хотите использовать внутри jupyter-notebook
```
## Пример

In [2]:
!nbqa ruff Linters.ipynb

E401 [*] Multiple imports on one line
 --> Linters.ipynb:2:1
  |
1 | # %%NBQA-CELL-SEP17da60
2 | import os, sys
  | ^^^^^^^^^^^^^^
3 | import sympy as sp
  |
help: Split imports

F401 [*] `os` imported but unused
 --> Linters.ipynb:2:8
  |
1 | # %%NBQA-CELL-SEP17da60
2 | import os, sys
  |        ^^
3 | import sympy as sp
  |
help: Remove unused import

F401 [*] `sys` imported but unused
 --> Linters.ipynb:2:12
  |
1 | # %%NBQA-CELL-SEP17da60
2 | import os, sys
  |            ^^^
3 | import sympy as sp
  |
help: Remove unused import

F401 [*] `sympy` imported but unused
 --> Linters.ipynb:3:17
  |
1 | # %%NBQA-CELL-SEP17da60
2 | import os, sys
3 | import sympy as sp
  |                 ^^
  |
help: Remove unused import: `sympy`

Found 4 errors.
[*] 4 fixable with the `--fix` option.


## Рассмотрим вывод:
- E401 → Несколько импортов на одной строке
- F401 → Модуль os, sys, Sympy импортирован, но не используется.
Найдено 4 ошибки, все помечены как **[*] fixable**. Тоесть их можно исправить запустив с флагом --fix. После такого, код не будет выдавать ошибок.
##### Заметьте, его не смутила длина строки больше 79 символов:
За правило про длину строки отвечает E501 (line too long).
У Ruff это правило выключено по умолчанию, чтобы не мешать людям, которые используют автоформаттеры вроде Black (у него длина строки 88 символов).
Поэтому если мы не включим E501 явно в файле,**pyproject.toml** Ruff будет молчать на любые длинные строки.
```bash
[tool.ruff]
line-length = 79          # нужный нам лимит
```

# Подробный разбор Pylint
Pylint — очень «строгий» линтер. Он анализирует код через собственное AST (astroid), пытаясь инферировать типы и следить за взаимодействием компонентов. Проверки Pylint охватывают: ошибки (undefined names, неправильные вызовы и т.д.), соблюдение соглашений (PEP8-стиль, нейминг), сложности функций, design-паттерны (большие классы/модули), дублирование кода и др. Он выводит оценки (баллы) и подробные описания.

## Философия и подход
- Монолитный инструмент с глубоким анализом
- Поиск потенциальных ошибок и проблем
- Оценка кода по 10-балльной шкале

## Преимущества
- Благодаря инферированию кода, Pylint умеет обнаруживать сложные баги: например, неверное наследование, несоответствие интерфейсов, потенциальные ошибки приведения типов
- Детальные отчеты и рекомендации
- Поиск сложных архитектурных проблем
- Расширенный вывод типов
- Поддержка плагинов (чекеров)

## Недостатки
- Pylint существенно медленнее других линтеров из-за сложного анализа
- Избыточная строгость для некоторых проектов
- Сложность настройки
- Нет автоматического исправления

## Установка и настройка
```bash
pip install pylint
```
Для настойки мы создаем конфигурационный файл в котором и будем проводить все настройки:
```bash
pylint --generate-rcfile > NAME.pylintrc
```
## Использование 
```bash
!nbqa pylint /path/to/file.ipynb #Внути ноутбука
pylint /path/to/file.ipynb
```

## Классификация ошибок
- E (Error) — Ошибка, которая помешает выполнению кода.
- W (Warning) — Предупреждение, потенциальная проблема.
- C (Convention) — Нарушение соглашения о стиле кода (PEP 8).
- R (Refactor) — Рекомендация по рефакторингу.
- F (Fatal) — Критическая ошибка, которая остановила анализ Pylint.

In [4]:
!nbqa pylint Linters.ipynb

************* Module Linters
Linters.ipynb:cell_1:16:0: C0301: Line too long (134/100) (line-too-long)
Linters.ipynb:cell_1:0:0: C0114: Missing module docstring (missing-module-docstring)
Linters.ipynb:cell_1:0:0: C0103: Module name "Linters" doesn't conform to snake_case naming style (invalid-name)
Linters.ipynb:cell_1:1:0: C0410: Multiple imports on one line (os, sys) (multiple-imports)
Linters.ipynb:cell_1:5:0: C0116: Missing function or method docstring (missing-function-docstring)
Linters.ipynb:cell_1:5:0: R1711: Useless return at end of function or method (useless-return)
Linters.ipynb:cell_1:12:0: C0103: Constant name "unused_var" doesn't conform to UPPER_CASE naming style (invalid-name)
Linters.ipynb:cell_1:15:0: C0116: Missing function or method docstring (missing-function-docstring)
Linters.ipynb:cell_1:1:0: W0611: Unused import os (unused-import)
Linters.ipynb:cell_1:1:0: W0611: Unused import sys (unused-import)
Linters.ipynb:cell_1:2:0: W0611: Unused sympy imported as sp (u

## Посмотрим на то какие ошибки он нам выдал:
- C0301 - Строка слишком длинная, превышает лимит символов  
- C0114 - Нет docstring модуля
- C0103 - Имя модуля/константы не соответствует snake_case / UPPER_CASE  
- C0410 - Несколько импортов на одной строке  
- C0116 - Нет docstring функции или метода
- R1711 - Лишний return в конце функции  
- W0611 - Импорт не используется

## Как мы видим он оценил наш код на 0/10.
Это одна из особенностей pylint. Анализ начинается с идеальной оценки 10.00/10. За каждое найденное нарушение (предупреждение, ошибка, замечание по стилю) из этой оценки вычитается определенное количество баллов.

## Сравним вывод ошибок разными линтерами
#### Flake8
```
Linters.ipynb:cell_1:1:10: E401 multiple imports on one line
```
##### Плюсы:
- Очень кратко и компактно.
- Сразу видно файл, строку, символ и код ошибки.
##### Минусы:
- Нет контекста строки кода, придётся открывать файл.
- Нет подсказки, как исправить.
#### Ruff
```
E401 [*] Multiple imports on one line
 --> Linters.ipynb:2:1
  |
1 | # %%NBQA-CELL-SEP17da60
2 | import os, sys
  | ^^^^^^^^^^^^^^
3 | import sympy as sp
  |
help: Split imports
```
##### Плюсы:
- Показан контекст (несколько строк кода).
- Есть точная подсветка проблемного места.
- Даёт подсказку как исправить (help: Split imports).
##### Минусы:
- Вывод более громоздкий, в большом проекте итоговый вывод может быть слишком многословный.
#### Pylint
```
Linters.ipynb:cell_1:1:0: C0410: Multiple imports on one line (os, sys) (multiple-imports)
```
##### Плюсы:
- Показывает и код ошибки (C0410), и человекочитаемое описание.
- Даёт уточнение прямо в скобках (os, sys), то есть показывает, какие именно импорты объединены.

##### Минусы:
- Не подсвечивает строку.
- Нет примера исправления.

# Сравнение подавления проверок в линтерах

## Flake8
- Используйте комментариий `# noqa`
- Можно отключить все проверки для строки или только конкретное правило

**Примеры:**
```python
import os, sys  # noqa        # отключает все ошибки на строке
import os, sys  # noqa: E401  # отключает только E401
```

**Глобальная конфигурация** (в `setup.cfg`):
```cfg
[flake8]
ignore = E401
```

## Ruff
- Использует `# noqa` и `# noqa: <CODE>` (аналогично Flake8)
- Поддерживает `# ruff: noqa` и более тонкие директивы
- Можно отключать для блока с помощью `# ruff: noqa` в начале
- Есть поддержка per-file ignores

**Примеры:**
```python
import os, sys  # noqa: E401        # подавление конкретного правила
import os, sys  # ruff: noqa        # отключение всех проверок для строки
```

**Конфигурация в `pyproject.toml`:**
```toml
[tool.ruff]
ignore = ["E401"]
```

## Pylint

- Использует комментарии `# pylint: disable=<rule-name>`
- Можно отключать на строку, функцию, класс или даже целый файл

**Примеры:**

*Для одной строки:*
```python
import os, sys  # pylint: disable=multiple-imports
```

*Для блока:*
```python
# pylint: disable=multiple-imports
import os, sys
import math, random
# pylint: enable=multiple-imports
```

*Для всего файла:*
```python
# pylint: disable=multiple-imports
```

**Конфигурация в `.pylintrc`:**
```ini
[MESSAGES CONTROL]
disable=multiple-imports
```

# Вывод
Ruff является наиболее предпочтительным выбором для большинства современных Python-проектов благодаря своей исключительной производительности (в 10–100 раз быстрее конкурентов), комплексному подходу «всё-в-одном», простоте настройки и активному развитию. Особенно удобен для локальной разработки.

Flake8 с плагинами рекомендуется для больших проектов, требующих тонкой настройки и доступа к специализированным плагинам. Он остаётся актуальным благодаря своей гибкости и огромной экосистеме. Отлично подходит для CI-пайплайнов, где важна компактность отчётов.

Pylint стоит выбирать для долгоживущих проектов, где критически важен глубокий статический анализ, надёжность и выявление более сложных ошибок стиля и логики. Хорош в случаях, когда крайне важна стабильность и безопасность кода.